# Data Preprocessing

author: Melih Kacaman
date: 05-10-2022

# Constants

In [3]:
import warnings
warnings.filterwarnings("ignore")

DATASET1_ODBC = 'PROTO.Melih.Dataset1_SimilarProducts'

## Connect to the MSSQL Server

In [4]:
from common.sql_helper import custom_query
import pandas as pd

query = f'select * from {DATASET1_ODBC}'
result = custom_query(query=query)

result.head()

,WebSecenek,MarkaKodu,CinsiyetKodu,UrunGrubuKodu,Renk,cUrl,UrlNo
0,30028252-880,05,1,033,880,https://resim.aydinli.com.tr/05/1/033/S3002825...,1
1,30028252-880,05,1,033,880,https://resim.aydinli.com.tr/05/1/033/S3002825...,2
2,30028262-591,05,1,033,591,https://resim.aydinli.com.tr/05/1/033/S3002826...,1
3,30028262-591,05,1,033,591,https://resim.aydinli.com.tr/05/1/033/S3002826...,2
4,30028263-001,05,1,033,001,https://resim.aydinli.com.tr/05/1/033/S3002826...,1


# Download Images

In [30]:
class_names = result[['CinsiyetKodu', 'UrunGrubuKodu', 'Renk']].drop_duplicates()
class_names.head()

,CinsiyetKodu,UrunGrubuKodu,Renk
0,1,033,880
2,1,033,591
4,1,033,001
8,1,033,400
10,1,033,790


In [ ]:
import requests

def save_image(image_url, path):
    img_data = requests.get(image_url).content
    with open(path, 'wb') as handler:
        handler.write(img_data)

In [103]:
import os

def foldering(bran_id, dataset_name):
    path = f'/datasets/{dataset_name}'

    if not os.path.isdir(path):
        os.mkdir(path)

    path = os.path.join(path, f'/brand_{bran_id}')

    if not os.path.isdir(path):
        os.mkdir(path)



    return


In [104]:
res = foldering('08')
res.CinsiyetKodu

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/datasets/dataset1/brand_08'

In [88]:
for index, row in class_names.iterrows():
    _class = row['CinsiyetKodu'] + '/'+ row['UrunGrubuKodu'] + '/' + row['Renk']
    folder = path + '/' + _class

    res = result.query(f'MarkaKodu == "{bran_id}" and CinsiyetKodu == "{row.CinsiyetKodu}" and UrunGrubuKodu == "{row.UrunGrubuKodu}" and Renk == "{row.Renk}"')

    # if exists any data
    if res.shape[0] > 0:
        # check the folder exist
        if os.path.isdir(folder):
            save_image()
        else:

## For USPA

## For PC

## For Cacharel